# Swim relay problem (Van Roy and Mason)
The coach of a swim team needs to assign swimmers to a 200-yard medley relay team to compete in a tournament. The problem is that his best swimmers are good in more than one stroke, so it's not clear which swimmer to assign to which stroke. Here are the best times for each swimmer:

|Stroke       | Carl | Chris | David | Tony | Ken  |
|-------------|:----:|:-----:|:-----:|:----:|:----:|
|Backstroke   | 37.7 | 32.9  | 33.8  | 37.0 | 35.4 |
|Breaststroke | 43.4 | 33.1  | 42.2  | 34.7 | 41.8 |
|Butterfly    | 33.3 | 28.5  | 38.9  | 30.4 | 33.6 |
|Freestyle    | 29.2 | 26.4  | 29.6  | 28.5 | 31.1 |


### Solution (Classic Transportation)

In [48]:
using JuMP, HiGHS, NamedArrays

strokes = [ :backstroke, :breaststroke, :butterfly, :freestyle ]
names = [ :Carl, :Chris, :David, :Tony, :Ken ]

raw = [ 37.7 32.9 33.8 37.0 35.4
        43.4 33.1 42.2 34.7 41.8
        33.3 28.5 38.9 30.4 33.6
        29.2 26.4 29.6 28.5 31.1 ]

times = NamedArray( raw, (strokes,names), ("stroke","name"))

m = Model(HiGHS.Optimizer)

@variable(m, x[strokes,names] >= 0)

# each swimmer swims at most one event
@constraint(m, a[j in names], sum(x[i,j] for i in strokes) <= 1 )

# each event has exactly one swimmer
@constraint(m, b[i in strokes], sum(x[i,j] for j in names) == 1 )

@objective(m, Min, sum( x[i,j]*times[i,j] for i in strokes, j in names ) )

#print(m)
optimize!(m)

Presolving model
9 rows, 20 cols, 40 nonzeros
9 rows, 20 cols, 40 nonzeros
Presolve : Reductions: rows 9(-0); columns 20(-0); elements 40(-0)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 4(4) 0s
          8     1.2620000000e+02 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 8
Objective value     :  1.2620000000e+02
HiGHS run time      :          0.00


In [43]:
assignment = NamedArray( [ (value(x[i,j])) for i in strokes, j in names ], (strokes, names), ("stroke","name"))

4×5 Named Matrix{Float64}
stroke ╲ name │  :Carl  :Chris  :David   :Tony    :Ken
──────────────┼───────────────────────────────────────
:backstroke   │    0.0     0.0     1.0     0.0     0.0
:breaststroke │    0.0     0.0     0.0     1.0     0.0
:butterfly    │    0.0     1.0     0.0    -0.0     0.0
:freestyle    │    1.0    -0.0     0.0     0.0     0.0

### Solution (MCNFP)

In [45]:
#Problem is unbalanced, need a dummy node with 0 cost
#vec(matrix) creates a vector from a matrix
costs = vec([raw;
         0 0 0 0 0]')
b = [1; 1; 1; 1; 1; -1; -1; -1; -1; -1]

#Form incidence matrix, first 5 rows are swimmers, last 5 are strokes (1 dummy node)
numswimmers = 5
numstrokes = 5
numedges = numswimmers*numstrokes
A = zeros(numswimmers+numstrokes,numedges);
edgenum = 1
for i in 1:numswimmers
    for j in 1:numstrokes
        #Edge leaves swimmer i
        A[i,edgenum] = 1
        #Edge enters stroke j
        A[numswimmers+j,edgenum] = -1
        #Increase edge number
        edgenum = edgenum+1
    end
end
A

10×25 Matrix{Float64}:
  1.0   1.0   1.0   1.0   1.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      1.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   1.0   1.0   1.0   1.0   1.0
 -1.0   0.0   0.0   0.0   0.0  -1.0  …   0.0  -1.0   0.0   0.0   0.0   0.0
  0.0  -1.0   0.0   0.0   0.0   0.0      0.0   0.0  -1.0   0.0   0.0   0.0
  0.0   0.0  -1.0   0.0   0.0   0.0      0.0   0.0   0.0  -1.0   0.0   0.0
  0.0   0.0   0.0  -1.0   0.0   0.0      0.0   0.0   0.0   0.0  -1.0   0.0
  0.0   0.0   0.0   0.0  -1.0   0.0     -1.0   0.0   0.0   0.0   0.0  -1.0

In [46]:
m2 = Model(HiGHS.Optimizer)

@variable(m2, x[1:numedges] >= 0)

#Use incidence matrix to define constraint
@constraint(m2, flow, A*x .== b)

@objective(m2, Min, sum(x[i]*costs[i] for i in 1:numedges))

optimize!(m2)

Presolving model
10 rows, 25 cols, 50 nonzeros
9 rows, 25 cols, 45 nonzeros
Presolve : Reductions: rows 9(-1); columns 25(-0); elements 45(-5)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 9(9) 0s
         12     1.2620000000e+02 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 12
Objective value     :  1.2620000000e+02
HiGHS run time      :          0.01


In [41]:
xout = value.(x)
xnodummy = xout[1:20]
xmat = reshape(xnodummy, (numswimmers,numstrokes-1))
assignment = NamedArray( xmat', (strokes, names), ("stroke","name"))

4×5 Named LinearAlgebra.Adjoint{Float64, Matrix{Float64}}
stroke ╲ name │  :Carl  :Chris  :David   :Tony    :Ken
──────────────┼───────────────────────────────────────
:backstroke   │    0.0     0.0     1.0     0.0     0.0
:breaststroke │    0.0     0.0     0.0     1.0     0.0
:butterfly    │    0.0     1.0     0.0    -0.0     0.0
:freestyle    │    1.0    -0.0     0.0     0.0     0.0